In [1]:
from keras.applications.vgg16 import VGG16
from keras.utils.vis_utils import plot_model
# prepare the image for the VGG model
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras import optimizers
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import os
from sklearn.model_selection import train_test_split

C:\Users\sitandon\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [34]:
img_rows, img_cols, img_channel = 224, 224, 3
productMatrix = pd.read_csv("./data/train/ProductMatrix")

totalClasses = len(np.unique(productMatrix["TargetCode"]))
train,test = train_test_split(productMatrix,test_size = .1)

In [35]:
#train.groupby("TargetCode").count().sort_values(by = "productid",ascending = False)
submissionDFUsers = pd.read_csv("./data/test.csv")
txnDF = pd.read_csv("./data/train/train.csv")
productAggData = txnDF.groupby(["productid","UserId"])["Quantity"].count().reset_index()
productAggData.columns = ["productid","UserId","PurchaseCount"]

In [36]:
batch_size = 32

In [5]:
def imageLoader(data, batch_size):
    
    L = len(data)
    
    def loadImageAndTarget(df):
        imagesList = []
        targetList = []
        
        for row in df.values:
            imagePath = os.path.join("./data/train/images",str(row[0]) + ".jpg")
            targets = np.zeros(shape = (1,totalClasses),dtype = "int")
            targets[0,row[1]]= 1
            image = load_img(imagePath, target_size=(img_rows, img_cols))
            # convert the image pixels to a numpy array
            image = img_to_array(image)
            imagesList.append(image)
            targetList.append(targets)
        return imagesList,targetList
        
        
    #this line is just to make the generator infinite, keras needs that    
    while True:

        batch_start = 0
        batch_end = batch_size

        while batch_start < L:
            limit = min(batch_end, L)
            
            X,Y = loadImageAndTarget(data[batch_start:limit])
            
            X=np.asarray(X).reshape(-1,img_rows, img_cols,img_channel)
            Y = np.asarray(Y).reshape(-1,totalClasses)
            yield (X,Y) #a tuple with two numpy arrays with batch_size samples     

            batch_start += batch_size   
            batch_end += batch_size

In [11]:
#--------Define the model
model = VGG16(weights=None,include_top=False,input_shape=(img_rows, img_cols, img_channel),classes = totalClasses)
model.load_weights("./data/train/weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5",by_name=True)

for layer in model.layers:
    layer.trainable = False
    
keras_input = Input(shape=(img_rows,img_cols,img_channel),name = 'image_input')

#Use the generated model 
output_vgg16_conv = model(keras_input)

x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(1024, activation='relu', name='fc1')(x)
x = Dense(1024, activation='relu', name='fc2')(x)
x = Dense(totalClasses, activation='softmax', name='predictions')(x)

#Create your own model 
my_model = Model(input=keras_input , output=x)

optim = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
#optim = optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=1e-6)
#optim = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-6, amsgrad=False)
my_model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
#my_model.summary()

C:\Users\sitandon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("im..., outputs=Tensor("pr...)`


In [39]:
trainGenerator = imageLoader(train,batch_size)
testGenerator = imageLoader(test,batch_size)
fullDataGenerator = imageLoader(productMatrix,batch_size)
my_model.fit_generator(trainGenerator
                    ,steps_per_epoch= train.shape[0] // batch_size
                    , epochs=10
                    , validation_data = testGenerator
                    , validation_steps = test.shape[0] // batch_size
                    , verbose = 1
                   )

Epoch 1/10
85/85 [==============================] - 60s 706ms/step - loss: 3.0848 - acc: 0.8037 - val_loss: 3.1208 - val_acc: 0.8021
Epoch 2/10
85/85 [==============================] - 63s 747ms/step - loss: 3.1747 - acc: 0.8007 - val_loss: 3.1809 - val_acc: 0.7970
Epoch 3/10
85/85 [==============================] - 59s 698ms/step - loss: 3.1473 - acc: 0.7996 - val_loss: 3.1517 - val_acc: 0.7934
Epoch 4/10
85/85 [==============================] - 61s 720ms/step - loss: 3.0747 - acc: 0.8073 - val_loss: 3.1075 - val_acc: 0.8007
Epoch 5/10
85/85 [==============================] - 63s 742ms/step - loss: 3.0229 - acc: 0.8080 - val_loss: 3.1963 - val_acc: 0.7970
Epoch 6/10
85/85 [==============================] - 64s 748ms/step - loss: 3.0416 - acc: 0.8047 - val_loss: 2.7931 - val_acc: 0.8155
Epoch 7/10
85/85 [==============================] - 65s 761ms/step - loss: 2.9608 - acc: 0.8110 - val_loss: 3.0539 - val_acc: 0.7970
Epoch 8/10
85/85 [==============================] - 66s 772ms/step - 

In [40]:
# With a second Model
intermediate_model = Model(inputs=my_model.layers[0].input, 
                              outputs=my_model.get_layer("fc2").output)
output = intermediate_model.predict_generator(fullDataGenerator
                                     , steps= (productMatrix.shape[0] // batch_size) + 1
                                    )

from sklearn.metrics.pairwise import cosine_similarity
productSimilarityMatrix = cosine_similarity(output)

In [45]:
productMatrix.head()

,productid,TargetCode
0,11139192,6
1,11139194,6
2,11139524,8
3,11139560,8
4,11139588,6


In [48]:
tmp = pd.concat([productMatrix[["productid"]]
           ,pd.DataFrame(productSimilarityMatrix)],axis = 1)

mergedDF = productAggData[["UserId","productid"]].merge(tmp, how = "inner",on = "productid", suffixes = "_y")
cols = mergedDF.columns[2:]
aggMergedDF = mergedDF.groupby("UserId")[cols].agg(['count','sum'])
aggMergedDF.columns = [str(col[0]) + "_" + col[1] for col in aggMergedDF.columns]
for col in cols:
    aggMergedDF[str(col) + "_sum"] = aggMergedDF[str(col) + "_sum"]/ aggMergedDF["0_count"]
aggMergedDF = aggMergedDF[[str(col) + "_sum" for col in cols]]
aggMergedDF.columns = [str(col).replace("_sum","") for col in aggMergedDF.columns]

simArrPerUser = np.asarray(aggMergedDF)

In [53]:
aggMergedDF.shape

(27778, 3026)

In [41]:
print(output.shape, productMatrix.shape)

(3026, 1024) (3026, 2)


In [16]:
np.argsort(-1 * similarity[0,:])

array([   0, 1864,    1, ...,  906, 2842,  846], dtype=int64)

In [ ]:
x = -1 * np.array([[1,0,2],[4,1,0]])
tmp = pd.DataFrame()
#np.argsort(x,axis = 1)
for index in range(x.shape[0]):
    tmp = pd.concat([tmp,pd.DataFrame({"Similarity":[','.join(str(a) for a in x[index,:])]})],axis = 0)
tmp

In [17]:
productMatrix.loc[[0,1864,1]]

,productid,TargetCode
0,11139192,6
1864,12657628,5
1,11139194,6


In [69]:
def recommendProductsForUserId(UserID, totalRecommendations):
    #UserID = 26784
    global simArrPerUser,aggMergedDF,productAggData

    UserSpecificProducts = set(productAggData[productAggData["UserId"] == UserID]["productid"])
    
    allProducts = productMatrix[["productid"]].values
    UserIDs = aggMergedDF.index.values
    UserIDIndex = np.where(UserIDs == UserID)
    
    productIds = []
    productSortedSimilarity = np.argsort(-1 * simArrPerUser[UserIDIndex,:]).reshape(1,-1)
    
    #pdb.set_trace()
    for index in range(productSortedSimilarity.shape[1]):
        productIndex = productSortedSimilarity[0,index]
        if allProducts[productIndex][0] not in UserSpecificProducts:
            productIds.append(allProducts[productIndex][0])
    
    
    return productIds[0:totalRecommendations]

In [71]:
submission = pd.DataFrame()
for user in submissionDFUsers["UserId"].values:
    recommendations = "[" + ",".join(str(prod) for prod in recommendProductsForUserId(user,10)) + "]"
    submission = pd.concat([submission,pd.DataFrame({"UserId":[user],"product_list":[recommendations]})], axis = 0)

submission.to_csv("./Data/submission.csv",index= False)

In [70]:
recommendProductsForUserId(2,10)

[13039832,
 12658066,
 11660062,
 12657954,
 11659884,
 13039834,
 11660210,
 11481030,
 12407706,
 12407188]